# ODR Implementation
- Model specification, Fitting procedures, Result analysis
- Real examples: Nonlinear models, Multi-parameter fits

In [1]:
import numpy as np
from scipy import odr
import matplotlib.pyplot as plt
print('ODR implementation module loaded')

ODR implementation module loaded


## Model Specification

**Function signature**: `func(B, x)`
- **B**: Parameter array (what we're fitting)
- **x**: Independent variable(s)
- **Returns**: Predicted y values

**Important**:
- First argument must be parameters (B)
- Second argument must be independent variable (x)
- Must return array of same shape as x
- Can be any mathematical function

In [2]:
print('Common Model Functions\n')
print('='*60)

# 1. Linear model
def linear(B, x):
    '''y = B[0]*x + B[1]'''
    return B[0] * x + B[1]

# 2. Quadratic model
def quadratic(B, x):
    '''y = B[0]*x² + B[1]*x + B[2]'''
    return B[0] * x**2 + B[1] * x + B[2]

# 3. Exponential model
def exponential(B, x):
    '''y = B[0] * exp(B[1] * x)'''
    return B[0] * np.exp(B[1] * x)

# 4. Power law
def power_law(B, x):
    '''y = B[0] * x^B[1]'''
    return B[0] * x**B[1]

# 5. Gaussian
def gaussian(B, x):
    '''y = B[0] * exp(-((x-B[1])/B[2])²)'''
    return B[0] * np.exp(-((x - B[1]) / B[2])**2)

print('Model library defined:')
print('  1. Linear: y = a*x + b')
print('  2. Quadratic: y = a*x² + b*x + c')
print('  3. Exponential: y = a*exp(b*x)')
print('  4. Power law: y = a*x^b')
print('  5. Gaussian: y = a*exp(-((x-μ)/σ)²)')

Common Model Functions

Model library defined:
  1. Linear: y = a*x + b
  2. Quadratic: y = a*x² + b*x + c
  3. Exponential: y = a*exp(b*x)
  4. Power law: y = a*x^b
  5. Gaussian: y = a*exp(-((x-μ)/σ)²)


## Real Example: Radioactive Decay

**Physical process**: Exponential decay
**Model**: N(t) = N₀ * exp(-λt)
**Problem**: Both time and count measurements have errors

This is a classic ODR application because:
- Time measurements have uncertainty
- Counts have Poisson noise
- Need accurate decay constant (λ)

In [3]:
print('\nRadioactive Decay Fitting')
print('='*60)

np.random.seed(42)

# True physical parameters
N0_true = 1000  # Initial particle count
lambda_true = 0.1  # Decay constant (1/time)

# Time measurements (with timing error ±0.2)
t_true = np.array([0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20])
t_obs = t_true + np.random.randn(len(t_true)) * 0.2

# Count measurements (Poisson-like noise)
N_true = N0_true * np.exp(-lambda_true * t_true)
N_obs = N_true + np.random.randn(len(N_true)) * np.sqrt(N_true)

print('Experimental setup:')
print(f'  Measurements: {len(t_obs)} time points')
print(f'  Time range: {t_obs.min():.1f} to {t_obs.max():.1f} units')
print(f'  Count range: {N_obs.max():.0f} to {N_obs.min():.0f}')
print(f'  Timing uncertainty: ±0.2 time units')
print(f'  Count uncertainty: Poisson (√N)\n')
print('True parameters:')
print(f'  N₀ = {N0_true}')
print(f'  λ = {lambda_true}')


Radioactive Decay Fitting
Experimental setup:
  Measurements: 11 time points
  Time range: 0.1 to 19.9 units
  Count range: 985 to 133
  Timing uncertainty: ±0.2 time units
  Count uncertainty: Poisson (√N)

True parameters:
  N₀ = 1000
  λ = 0.1


In [4]:
# Define decay model
def decay_model(B, t):
    '''Exponential decay: N = N₀ * exp(-λ*t)'''
    N0, lam = B
    return N0 * np.exp(-lam * t)

# Set up ODR
model = odr.Model(decay_model)
data = odr.RealData(t_obs, N_obs, sx=0.2, sy=np.sqrt(N_obs))
odr_obj = odr.ODR(data, model, beta0=[1000, 0.1])  # Initial guess
result = odr_obj.run()

N0_fit, lambda_fit = result.beta
N0_err, lambda_err = result.sd_beta

print('\nFitting Results:')
print('='*60)
print('Fitted parameters:')
print(f'  N₀ = {N0_fit:.1f} ± {N0_err:.1f}')
print(f'  λ = {lambda_fit:.4f} ± {lambda_err:.4f}')
print()
print('True parameters:')
print(f'  N₀ = {N0_true}')
print(f'  λ = {lambda_true:.4f}')
print()
print('Derived quantity:')
half_life = np.log(2) / lambda_fit
half_life_err = np.log(2) * lambda_err / lambda_fit**2
print(f'  Half-life: {half_life:.2f} ± {half_life_err:.2f} time units')
print()
print('✓ Excellent agreement with true values!')


Fitting Results:
Fitted parameters:
  N₀ = 978.8 ± 23.5
  λ = 0.1002 ± 0.0027

True parameters:
  N₀ = 1000
  λ = 0.1000

Derived quantity:
  Half-life: 6.92 ± 0.19 time units

✓ Excellent agreement with true values!


## Result Analysis

**Key output attributes**:
- `beta`: Fitted parameter values
- `sd_beta`: Standard errors (uncertainties)
- `cov_beta`: Covariance matrix
- `res_var`: Residual variance (goodness of fit)
- `sum_square`: Sum of squared residuals
- `stopreason`: Convergence information

These provide complete statistical information about the fit.

In [5]:
print('\nDetailed Result Analysis')
print('='*60)

print('Fit Quality Metrics:')
print(f'  Residual variance: {result.res_var:.6f}')
print(f'  Sum of squares: {result.sum_square:.2f}')
print(f'  Degrees of freedom: {len(t_obs) - len(result.beta)}')
print(f'  Reduced chi-square: {result.sum_square / (len(t_obs) - len(result.beta)):.4f}')
print()
print('Parameter Covariance Matrix:')
print(result.cov_beta)
print()
# Correlation coefficient
corr_matrix = result.cov_beta / np.outer(result.sd_beta, result.sd_beta)
print(f'Parameter correlation: {corr_matrix[0,1]:.4f}')
print('  (Shows how parameters are related)')
print()
print(f'Convergence info:')
print(f'  Status: {result.stopreason}')
print(f'  Info: {result.info}')


Detailed Result Analysis
Fit Quality Metrics:
  Residual variance: 0.918069
  Sum of squares: 8.26
  Degrees of freedom: 9
  Reduced chi-square: 0.9181

Parameter Covariance Matrix:
[[5.99879960e+02 5.25621114e-02]
 [5.25621114e-02 7.83958327e-06]]

Parameter correlation: 0.8349
  (Shows how parameters are related)

Convergence info:
  Status: ['Sum of squares convergence']
  Info: 1


## Real Example: Allometric Scaling

**Biology**: Relationship between body mass and length
**Model**: Mass ∝ Length^α (power law)
**Application**: Growth studies, comparative biology

Power law with α ≈ 3 suggests isometric scaling (volume relationship)

In [6]:
print('\nAllometric Scaling: Mass vs Length')
print('='*60)

np.random.seed(42)

# Simulated organism measurements
length = np.array([10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60])  # cm
mass_true = 0.05 * length**2.8  # True allometric relationship

# Add measurement errors
length_obs = length + np.random.randn(len(length)) * 1.0  # ±1 cm
mass_obs = mass_true + np.random.randn(len(mass_true)) * (0.1 * mass_true)  # ±10%

print('Dataset:')
print(f'  Number of specimens: {len(length)}')
print(f'  Length range: {length_obs.min():.1f} to {length_obs.max():.1f} cm')
print(f'  Mass range: {mass_obs.min():.1f} to {mass_obs.max():.1f} g')
print(f'  Length uncertainty: ±1.0 cm')
print(f'  Mass uncertainty: ±10% (proportional)')
print()
print('True relationship: Mass = 0.05 * Length^2.8')


Allometric Scaling: Mass vs Length
Dataset:
  Number of specimens: 11
  Length range: 10.5 to 59.5 cm
  Mass range: 30.1 to 4654.5 g
  Length uncertainty: ±1.0 cm
  Mass uncertainty: ±10% (proportional)

True relationship: Mass = 0.05 * Length^2.8


In [7]:
# Power law model
def power_law(B, x):
    '''Power law: y = B[0] * x^B[1]'''
    return B[0] * x**B[1]

model = odr.Model(power_law)
data = odr.RealData(length_obs, mass_obs, 
                     sx=1.0,                    # Length error
                     sy=0.1 * mass_obs)         # Mass error (proportional)
odr_obj = odr.ODR(data, model, beta0=[0.05, 2.5])  # Initial guess
result = odr_obj.run()

coeff, exponent = result.beta
coeff_err, exp_err = result.sd_beta

print('\nAllometric Fit Results:')
print('='*60)
print('Power law equation:')
print(f'  Mass = {coeff:.4f} * Length^{exponent:.3f}')
print()
print('Parameter uncertainties:')
print(f'  Coefficient: ±{coeff_err:.4f}')
print(f'  Exponent: ±{exp_err:.3f}')
print()
print('True parameters:')
print(f'  Coefficient: 0.0500')
print(f'  Exponent: 2.800')
print()
print('Biological interpretation:')
if 2.5 < exponent < 3.0:
    print(f'  Exponent {exponent:.3f} ≈ 3.0')
    print('  → Suggests near-isometric (3D) scaling')
    print('  → Consistent with geometric scaling laws')
else:
    print(f'  Exponent {exponent:.3f} deviates from 3.0')
    print('  → May indicate allometric scaling')


Allometric Fit Results:
Power law equation:
  Mass = 0.0259 * Length^2.956

Parameter uncertainties:
  Coefficient: ±0.0088
  Exponent: ±0.092

True parameters:
  Coefficient: 0.0500
  Exponent: 2.800

Biological interpretation:
  Exponent 2.956 ≈ 3.0
  → Suggests near-isometric (3D) scaling
  → Consistent with geometric scaling laws


## Fixed Parameters

**Use case**: Fix some parameters while fitting others
**Method**: Use `ifixb` parameter or modify model function

**Applications**:
- Theory predicts certain parameter values
- Constrain model to physical limits
- Reduce parameter space for stability

In [8]:
print('\nFixed Parameter Example')
print('='*60)

# Generate data
np.random.seed(42)
x = np.linspace(0, 10, 20)
y = 2 * x + 5 + np.random.randn(20) * 0.5

print('Scenario: Theoretical intercept known to be 5.0')
print(f'  Data points: {len(x)}')
print(f'  True relationship: y = 2*x + 5')
print(f'  Task: Fit slope only, intercept fixed at 5\n')


Fixed Parameter Example
Scenario: Theoretical intercept known to be 5.0
  Data points: 20
  True relationship: y = 2*x + 5
  Task: Fit slope only, intercept fixed at 5



In [9]:
# Method 1: Modify model function to fix parameter
def linear_fixed(B, x):
    '''Linear with intercept fixed at 5'''
    return B[0] * x + 5  # B[0] is slope, intercept is constant

model = odr.Model(linear_fixed)
data = odr.RealData(x, y)
odr_obj = odr.ODR(data, model, beta0=[2])  # Only one parameter now
result = odr_obj.run()

print('Fixed Parameter Fit:')
print('='*40)
print(f'  Slope: {result.beta[0]:.4f} ± {result.sd_beta[0]:.4f}')
print(f'  Intercept: 5.0000 (fixed)')
print()
print('Compare to free fit:')
def linear_free(B, x):
    return B[0] * x + B[1]
model2 = odr.Model(linear_free)
odr_obj2 = odr.ODR(data, model2, beta0=[2, 5])
result2 = odr_obj2.run()
print(f'  Slope: {result2.beta[0]:.4f} ± {result2.sd_beta[0]:.4f}')
print(f'  Intercept: {result2.beta[1]:.4f} ± {result2.sd_beta[1]:.4f}')
print()
print('Fixing parameters reduces uncertainties in remaining parameters')

Fixed Parameter Fit:
  Slope: 1.9641 ± 0.0165
  Intercept: 5.0000 (fixed)

Compare to free fit:
  Slope: 1.9115 ± 0.0288
  Intercept: 5.3568 ± 0.1682

Fixing parameters reduces uncertainties in remaining parameters


## Multi-Parameter Complex Models

**Example**: Dose-response curve (pharmacology)
**Model**: 4-parameter logistic

Used extensively in drug development and biochemistry.

In [10]:
print('\nComplex Model: 4-Parameter Logistic (Dose-Response)')
print('='*60)

def logistic4(B, x):
    '''4-parameter logistic: y = d + (a-d)/(1 + (x/c)^b)'''
    a, b, c, d = B  # min, Hill slope, IC50, max
    return d + (a - d) / (1 + (x / c)**b)

# Simulate dose-response data
np.random.seed(42)
dose = np.logspace(-2, 2, 15)  # 0.01 to 100
response_true = logistic4([0, 2, 1, 100], dose)
response_obs = response_true + np.random.randn(len(dose)) * 3

print('Model: 4-parameter logistic curve')
print('  Parameters:')
print('    a: Minimum response')
print('    b: Hill slope (steepness)')
print('    c: IC50 (half-maximal dose)')
print('    d: Maximum response')
print()
print(f'Dose-response data: {len(dose)} concentrations')
print(f'  Dose range: {dose.min():.2f} to {dose.max():.2f}')
print(f'  Response range: {response_obs.min():.1f} to {response_obs.max():.1f}')


Complex Model: 4-Parameter Logistic (Dose-Response)
Model: 4-parameter logistic curve
  Parameters:
    a: Minimum response
    b: Hill slope (steepness)
    c: IC50 (half-maximal dose)
    d: Maximum response

Dose-response data: 15 concentrations
  Dose range: 0.01 to 100.00
  Response range: -0.4 to 100.6


In [11]:
# Fit 4PL model
model = odr.Model(logistic4)
data = odr.RealData(dose, response_obs)
odr_obj = odr.ODR(data, model, beta0=[0, 2, 1, 100])
result = odr_obj.run()

a_fit, b_fit, c_fit, d_fit = result.beta

print('\nFitted Parameters:')
print('='*60)
print(f'  Minimum (a): {a_fit:.2f} ± {result.sd_beta[0]:.2f}')
print(f'  Hill slope (b): {b_fit:.3f} ± {result.sd_beta[1]:.3f}')
print(f'  IC50 (c): {c_fit:.3f} ± {result.sd_beta[2]:.3f}')
print(f'  Maximum (d): {d_fit:.2f} ± {result.sd_beta[3]:.2f}')
print()
print('True parameters: a=0, b=2, c=1, d=100')
print()
print('Interpretation:')
print(f'  Drug IC50: {c_fit:.3f} (concentration for 50% inhibition)')
print(f'  Hill slope: {b_fit:.3f} (cooperative binding)')
print(f'  Dynamic range: {a_fit:.1f} to {d_fit:.1f}')


Fitted Parameters:
  Minimum (a): -0.35 ± 1.58
  Hill slope (b): 2.847 ± 1.848
  IC50 (c): 0.957 ± 0.727
  Maximum (d): 96.95 ± 0.75

True parameters: a=0, b=2, c=1, d=100

Interpretation:
  Drug IC50: 0.957 (concentration for 50% inhibition)
  Hill slope: 2.847 (cooperative binding)
  Dynamic range: -0.3 to 97.0


/var/folders/d1/sfvddbbx4tg14yxq7trqmhyw0000gn/T/ipykernel_15448/4230129505.py:7: RuntimeWarning: invalid value encountered in power
  return d + (a - d) / (1 + (x / c)**b)


## Summary

### Model Functions:
```python
# Must have signature: func(B, x)
def my_model(B, x):
    # B[0], B[1], ... are parameters to fit
    return mathematical_expression
```

### Complete Fitting Workflow:
```python
# 1. Define model
model = odr.Model(model_func)

# 2. Prepare data (with errors)
data = odr.RealData(x, y, sx=x_err, sy=y_err)

# 3. Set up ODR with initial guess
odr_obj = odr.ODR(data, model, beta0=initial_guess)

# 4. Run optimization
result = odr_obj.run()

# 5. Extract results
params = result.beta          # Fitted values
errors = result.sd_beta        # Standard errors
covariance = result.cov_beta   # Covariance matrix
fit_quality = result.res_var   # Residual variance
```

### Result Attributes:
- `beta`: Parameter values  
- `sd_beta`: Standard errors  
- `cov_beta`: Covariance matrix  
- `res_var`: Residual variance  
- `sum_square`: χ²  
- `stopreason`: Convergence status  

### Best Practices:
✓ Provide good initial guesses (critical!)  
✓ Scale variables to similar magnitudes  
✓ Specify known error estimates  
✓ Check convergence status  
✓ Validate physically reasonable results  
✓ Report uncertainties with parameters